## Multiple Request Handling

In [ ]:
import requests
import json
import re

# API Keys
SPOONACULAR_API_KEY = "YOUR_SPOONACULAR_API_KEY"
WEATHERSTACK_API_KEY = "YOUR_WEATHERSTACK_API_KEY"
FIREWORKS_API_KEY = "YOUR_FIREWORKS_API_KEY"

def analyze_query_and_get_actions(user_input):
    """Enhanced LLM function to detect and handle multiple queries"""

    headers = {
        "Authorization": f"Bearer {FIREWORKS_API_KEY}",
        "Content-Type": "application/json"
    }

    prompt = f"""You are a JSON response generator. Analyze the user query and return ONLY valid JSON.

User Query: "{user_input}"

Available functions:
1. search_recipes(query, number=2) - Search for recipes
2. search_by_ingredients(ingredients) - Find recipes by ingredients
3. get_random_recipe() - Get random recipe
4. search_restaurants(query) - Search restaurants
5. get_nutrition_info(food_item) - Get nutrition facts
6. get_weather_data(location) - Get current weather

Rules:
- Return ONLY the JSON object, no other text
- Use "single" for one request, "multiple" for multiple requests
- Always include "query_type" and "functions" fields
- For locations in India, add ", India" to location parameter
- Be flexible and interpret user intent broadly
- If unsure about exact parameters, make reasonable assumptions

Examples:
{{"query_type": "single", "functions": [{{"function": "get_weather_data", "params": {{"location": "Delhi, India"}}}}]}}
{{"query_type": "single", "functions": [{{"function": "search_recipes", "params": {{"query": "pasta", "number": 2}}}}]}}
{{"query_type": "multiple", "functions": [{{"function": "get_weather_data", "params": {{"location": "Tokyo, Japan"}}}}, {{"function": "search_recipes", "params": {{"query": "sushi", "number": 2}}}}]}}

JSON Response:"""

    json_data = {
        "model": "accounts/fireworks/models/llama-v3p1-8b-instruct",
        "prompt": prompt,
        "max_tokens": 300,
        "temperature": 0.1
    }

    try:
        response = requests.post("https://api.fireworks.ai/inference/v1/completions",
                               headers=headers, json=json_data)
        result = response.json().get("choices", [{}])[0].get("text", "").strip()

        print(f"[DEBUG] Raw LLM response: {repr(result)}")  # Debug line

        # More robust JSON extraction
        json_patterns = [
            r'\{[^{}]*"query_type"[^{}]*"functions"[^{}]*\[[^\]]*\][^{}]*\}',  # Complete structure
            r'\{.*?"functions".*?\[.*?\].*?\}',  # Focus on functions array
            r'\{.*?\}',  # Any JSON-like structure
        ]

        json_str = None
        for pattern in json_patterns:
            match = re.search(pattern, result, re.DOTALL)
            if match:
                json_str = match.group()
                break

        if not json_str:
            print(f"[ERROR] No JSON found in response: {result}")
            return None

        # Clean up common JSON issues
        json_str = json_str.strip()
        # Remove trailing commas before closing brackets/braces
        json_str = re.sub(r',(\s*[}\]])', r'\1', json_str)
        # Fix any double quotes issues
        json_str = re.sub(r'([{,]\s*)"([^"]+)"\s*:', r'\1"\2":', json_str)

        print(f"[DEBUG] Cleaned JSON: {json_str}")  # Debug line

        try:
            parsed_json = json.loads(json_str)
            print(f"[DEBUG] Parsed successfully: {parsed_json}")  # Debug line
            return parsed_json
        except json.JSONDecodeError as json_error:
            print(f"[ERROR] JSON parsing failed: {json_error}")
            print(f"[ERROR] Problematic JSON: {json_str}")
            return None

    except Exception as e:
        print(f"[ERROR] LLM analysis failed: {e}")
        return None

In [ ]:
def format_multiple_responses_with_llm(results_data, user_query):
    """Format multiple API responses into a single coherent reply"""
    headers = {
        "Authorization": f"Bearer {FIREWORKS_API_KEY}",
        "Content-Type": "application/json"
    }

    # Truncate data if too large
    results_str = json.dumps(results_data, indent=2)
    if len(results_str) > 3000:
        results_str = results_str[:3000] + "...[truncated]"

    prompt = f"""You have multiple API responses for a user's query. Format them into ONE cohesive, friendly response.

User's Question: "{user_query}"
API Results: {results_str}

Instructions:
- Combine all responses into ONE conversational reply
- Use emojis and be friendly
- Separate different topics with line breaks or headers
- For recipes: Include title, time, key ingredients
- For weather: Include temperature, conditions, location
- Use **bold** for section headings
- Make it flow naturally - don't just list separate answers

Your combined response:"""

    json_data = {
        "model": "accounts/fireworks/models/llama-v3p1-8b-instruct",
        "prompt": prompt,
        "max_tokens": 400,
        "temperature": 0.4
    }

    try:
        response = requests.post("https://api.fireworks.ai/inference/v1/completions",
                               headers=headers, json=json_data)
        if response.status_code == 200:
            result = response.json().get("choices", [{}])[0].get("text", "").strip()
            return result if result else "Sorry, I couldn't generate a proper response."
        else:
            return "Sorry, there was an issue formatting the response."
    except Exception as e:
        return f"Sorry, I couldn't format the response properly. Error: {str(e)}"


In [ ]:
# API Functions
def search_recipes(query, number=2):
    """Search for recipes using Spoonacular API"""
    search_url = "https://api.spoonacular.com/recipes/complexSearch"
    search_params = {
        "apiKey": SPOONACULAR_API_KEY,
        "query": query,
        "number": number,
        "addRecipeInformation": True,
        "fillIngredients": True
    }

    try:
        search_response = requests.get(search_url, params=search_params)
        search_data = search_response.json()

        if "results" not in search_data or not search_data["results"]:
            return {"error": "No recipes found", "results": []}

        # Get detailed information for each recipe
        detailed_recipes = []
        for recipe in search_data["results"]:
            recipe_id = recipe["id"]
            detail_url = f"https://api.spoonacular.com/recipes/{recipe_id}/information"
            detail_params = {"apiKey": SPOONACULAR_API_KEY}

            detail_response = requests.get(detail_url, params=detail_params)
            if detail_response.status_code == 200:
                detailed_recipes.append(detail_response.json())

        return {"results": detailed_recipes}
    except Exception as e:
        return {"error": f"API call failed: {str(e)}"}

def search_by_ingredients(ingredients):
    """Find recipes based on available ingredients"""
    search_url = "https://api.spoonacular.com/recipes/findByIngredients"
    search_params = {
        "apiKey": SPOONACULAR_API_KEY,
        "ingredients": ingredients,
        "number": 3,
        "ranking": 1,
        "ignorePantry": True
    }

    try:
        search_response = requests.get(search_url, params=search_params)
        recipes_data = search_response.json()

        if not recipes_data:
            return {"error": "No recipes found with those ingredients"}

        # Get detailed information for each recipe
        detailed_recipes = []
        for recipe in recipes_data[:2]:  # Limit to 2 recipes
            recipe_id = recipe["id"]
            detail_url = f"https://api.spoonacular.com/recipes/{recipe_id}/information"
            detail_params = {"apiKey": SPOONACULAR_API_KEY}

            detail_response = requests.get(detail_url, params=detail_params)
            if detail_response.status_code == 200:
                detailed_recipe = detail_response.json()
                detailed_recipe['usedIngredients'] = recipe.get('usedIngredients', [])
                detailed_recipe['missedIngredients'] = recipe.get('missedIngredients', [])
                detailed_recipes.append(detailed_recipe)

        return {"results": detailed_recipes}
    except Exception as e:
        return {"error": f"API call failed: {str(e)}"}

def get_random_recipe():
    """Get a random recipe from Spoonacular API"""
    url = "https://api.spoonacular.com/recipes/random"
    params = {"apiKey": SPOONACULAR_API_KEY, "number": 1}

    try:
        response = requests.get(url, params=params)
        return response.json()
    except Exception as e:
        return {"error": f"API call failed: {str(e)}"}

def search_restaurants(query):
    """Search for restaurant information"""
    url = "https://api.spoonacular.com/food/restaurants/search"
    params = {
        "apiKey": SPOONACULAR_API_KEY,
        "query": query,
        "offset": 0,
        "number": 3
    }

    try:
        response = requests.get(url, params=params)
        return response.json()
    except Exception as e:
        return {"error": f"API call failed: {str(e)}"}

def get_nutrition_info(food_item):
    """Get nutritional information for a food item"""
    url = "https://api.spoonacular.com/recipes/guessNutrition"
    params = {"apiKey": SPOONACULAR_API_KEY, "title": food_item}

    try:
        response = requests.get(url, params=params)
        return response.json()
    except Exception as e:
        return {"error": f"API call failed: {str(e)}"}

def get_weather_data(location):
    """Get weather data from Weatherstack API"""
    url = "http://api.weatherstack.com/current"
    params = {
        "access_key": WEATHERSTACK_API_KEY,
        "query": location,
        "units": "m"  # Metric units
    }

    try:
        response = requests.get(url, params=params)
        data = response.json()

        if "error" in data:
            return {"error": f"Weather data not found for {location}"}

        return data
    except Exception as e:
        return {"error": f"API call failed: {str(e)}"}

def format_response_with_llm(api_data, user_query, api_function):
    """Use LLM to format response from API data"""
    headers = {
        "Authorization": f"Bearer {FIREWORKS_API_KEY}",
        "Content-Type": "application/json"
    }

    # Truncate API data if too large
    api_data_str = json.dumps(api_data, indent=2)
    if len(api_data_str) > 2500:
        api_data_str = api_data_str[:2500] + "...[truncated]"

    prompt = f"""Format this API response into a friendly, conversational reply with emojis.

CRITICAL: Your response should be ONLY the conversational reply - no notes, no code, no explanations, no extra text.

User's Question: "{user_query}"
API Function Called: {api_function}
API Response: {api_data_str}

Instructions:
- Be conversational and helpful
- Use appropriate emojis
- For recipes: Include title, cooking time, ingredients, and brief instructions
- For weather: Include temperature, conditions, and location
- For restaurants: Include name, cuisine type, and key details
- For nutrition: Include calories and key nutrients
- Use **bold** for headings
- If multiple results, separate with lines (──────)
- If no results found, give a helpful "not found" message
- STOP after the conversational response - no additional notes or code

Your response:"""

    json_data = {
        "model": "accounts/fireworks/models/llama-v3p1-8b-instruct",
        "prompt": prompt,
        "max_tokens": 300,
        "temperature": 0.4
    }

    try:
        response = requests.post("https://api.fireworks.ai/inference/v1/completions", headers=headers, json=json_data)
        if response.status_code == 200:
            result = response.json().get("choices", [{}])[0].get("text", "").strip()

            # Clean up the response - remove any extra content after the main response
            lines = result.split('\n')
            clean_lines = []

            for line in lines:
                # Stop at common patterns that indicate extra content
                if any(pattern in line.lower() for pattern in ['note:', 'here is', 'import ', 'def ', '```', 'api_key']):
                    break
                clean_lines.append(line)

            cleaned_result = '\n'.join(clean_lines).strip()
            return cleaned_result if cleaned_result else "Sorry, I couldn't generate a proper response."
        else:
            return f"Sorry, there was an issue formatting the response."
    except Exception as e:
        return f"Sorry, I couldn't format the response properly. Error: {str(e)}"



In [ ]:
def execute_functions_sequentially(functions_list, user_input):
    """Execute multiple functions one by one with progress feedback"""
    function_map = {
        "search_recipes": search_recipes,
        "search_by_ingredients": search_by_ingredients,
        "get_random_recipe": get_random_recipe,
        "search_restaurants": search_restaurants,
        "get_nutrition_info": get_nutrition_info,
        "get_weather_data": get_weather_data
    }

    # Function name mappings for user-friendly display
    function_display_names = {
        "search_recipes": "recipe search",
        "search_by_ingredients": "ingredient-based recipe search",
        "get_random_recipe": "random recipe",
        "search_restaurants": "restaurant search",
        "get_nutrition_info": "nutrition information",
        "get_weather_data": "weather data"
    }

    total_functions = len(functions_list)

    if total_functions > 1:
        print(f"Bot 🤖: I found {total_functions} requests in your message! Let me handle them one by one...")
        print("─" * 50)

    for i, func_info in enumerate(functions_list, 1):
        function_name = func_info["function"]
        params = func_info.get("params", {})
        display_name = function_display_names.get(function_name, function_name)

        # Show progress for multiple requests
        if total_functions > 1:
            print(f"\n📋 **Step {i}/{total_functions}:** Getting {display_name}...")

        if function_name not in function_map:
            print(f"Sorry, I don't know how to handle '{display_name}'")
            continue

        try:
            function = function_map[function_name]

            # Execute the function
            if params:
                api_result = function(**params)
            else:
                api_result = function()

            # Check for API errors
            if isinstance(api_result, dict) and "error" in api_result:
                print(f"{display_name.title()}: {api_result['error']}")
                continue

            # Format and display the result immediately
            response = format_response_with_llm(api_result, user_input, function_name)

            # Clean up response and display
            if response and response.strip():
                if total_functions > 1:
                    print(f"✅ **{display_name.title()}:**")
                print(f"{response}")
            else:
                print(f"Sorry, couldn't format the {display_name} properly.")

        except Exception as e:
            print(f"Error with {display_name}: {str(e)}")

        # Add separator between results (except for the last one)
        if total_functions > 1 and i < total_functions:
            print("\n" + "─" * 50)

    # Final message for multiple requests
    if total_functions > 1:
        print(f"\n🎉 **All done!** I've handled all {total_functions} of your requests!")

def execute_single_function(function_name, params):
    """Execute a single function (for backward compatibility)"""
    function_map = {
        "search_recipes": search_recipes,
        "search_by_ingredients": search_by_ingredients,
        "get_random_recipe": get_random_recipe,
        "search_restaurants": search_restaurants,
        "get_nutrition_info": get_nutrition_info,
        "get_weather_data": get_weather_data
    }

    if function_name not in function_map:
        return {"error": f"Unknown function: {function_name}"}

    try:
        function = function_map[function_name]
        if params:
            return function(**params)
        else:
            return function()
    except Exception as e:
        return {"error": f"Function execution failed: {str(e)}"}


In [ ]:
def main():
    print("🤖 **Enhanced Universal Assistant** - General LLM-Powered Version!")
    print("=" * 70)
    print("I can handle MULTIPLE requests using advanced AI understanding!")
    print("🍽️ **Food:** recipes, cooking tips, restaurant info, nutrition")
    print("🌦️ **Weather:** current conditions anywhere in the world")
    print("\n✨ **NEW:** Ask for multiple things at once in natural language!")
    print("Examples:")
    print("• 'Weather in Tokyo and show me sushi recipes'")
    print("• 'Give me pasta recipes and weather in Mumbai'")
    print("• 'Random recipe and nutrition info for pizza'")
    print("• 'Find restaurants near me and weather forecast'")
    print("\nType 'exit' to quit.\n")

    while True:
        user_input = input("You 🧑: ").strip()

        if user_input.lower() in ['exit', 'quit', 'bye', 'stop']:
            print("Bot 🤖: Goodbye! Have a great day! 🌟")
            break

        if not user_input:
            print("Bot 🤖: Please ask me something!")
            continue

        # Check API keys
        if not FIREWORKS_API_KEY or not SPOONACULAR_API_KEY or not WEATHERSTACK_API_KEY:
            print("Bot 🤖: ⚠️ Please configure your API keys first!")
            continue

        try:
            # Step 1: Analyze query using LLM only
            action_analysis = analyze_query_and_get_actions(user_input)

            if not action_analysis or "functions" not in action_analysis:
                print("Bot 🤖: I'm not sure how to help with that. Try asking about food or weather!")
                print("Bot 🤖: The AI couldn't understand your request. Please try rephrasing it.")
                continue

            functions_to_call = action_analysis["functions"]
            query_type = action_analysis.get("query_type", "single")

            # Optional debug info (comment out in production)
            # print(f"[DEBUG] Query type: {query_type}, Functions: {len(functions_to_call)}")

            # Step 2: Execute functions sequentially
            if len(functions_to_call) == 1:
                # Handle single request normally
                func_info = functions_to_call[0]
                function_name = func_info["function"]
                params = func_info.get("params", {})

                api_result = execute_single_function(function_name, params)

                if isinstance(api_result, dict) and "error" in api_result:
                    print(f"Bot 🤖: {api_result['error']} ")
                    continue

                response = format_response_with_llm(api_result, user_input, function_name)
                print(f"Bot 🤖: {response}")
            else:
                # Handle multiple requests with sequential processing
                execute_functions_sequentially(functions_to_call, user_input)

        except Exception as e:
            print("Bot 🤖: Oops! Something went wrong while processing your request.")
            print(f"Bot 🤖: Please try rephrasing your question or check your API keys.")
            print(f"[ERROR] {e}")

if __name__ == "__main__":
    main()

🤖 **Enhanced Universal Assistant** - General LLM-Powered Version!
I can handle MULTIPLE requests using advanced AI understanding!
🍽️ **Food:** recipes, cooking tips, restaurant info, nutrition
🌦️ **Weather:** current conditions anywhere in the world

✨ **NEW:** Ask for multiple things at once in natural language!
Examples:
• 'Weather in Tokyo and show me sushi recipes'
• 'Give me pasta recipes and weather in Mumbai'
• 'Random recipe and nutrition info for pizza'
• 'Find restaurants near me and weather forecast'

Type 'exit' to quit.

You 🧑: What's the weather in Delhi, india
[DEBUG] Raw LLM response: '{"query_type": "single", "functions": [{"function": "get_weather_data", "params": {"location": "Delhi, India"}}]}\n"""\n\nimport json\n\ndef search_recipes(query, number=2):\n    # This function is not implemented\n    pass\n\ndef search_by_ingredients(ingredients):\n    # This function is not implemented\n    pass\n\ndef get_random_recipe():\n    # This function is not implemented\n    p

## Single Request Handling

In [ ]:
import requests
import json
import re

# 🔑 API Keys
SPOONACULAR_API_KEY = "d21ccdf0b9cf4ba7b3bbbf7760ca4116"
WEATHERSTACK_API_KEY = "30885bb35b49451ae3a28ceca8a16a50"
FIREWORKS_API_KEY = "fw_3ZS6yxQoMod8jn5dTqXCyonR"

def analyze_query_and_get_action(user_input):
    """Use LLM to understand query and determine which API to call with parameters"""
    headers = {
        "Authorization": f"Bearer {FIREWORKS_API_KEY}",
        "Content-Type": "application/json"
    }

    prompt = f"""Analyze this user query and determine what API function to call. Respond with ONLY a JSON object.

User Query: "{user_input}"

Available API functions:
1. search_recipes(query, number=2) - Search for recipes
2. search_by_ingredients(ingredients) - Find recipes by ingredients (comma-separated)
3. get_random_recipe() - Get random recipe
4. search_restaurants(query) - Search restaurants
5. get_nutrition_info(food_item) - Get nutrition facts
6. get_weather_data(location) - Get current weather

IMPORTANT: If the query asks for multiple different things (like weather AND recipes), choose the FIRST/PRIMARY request only.

Examples:
- "pasta recipes" → {{"function": "search_recipes", "params": {{"query": "pasta", "number": 2}}}}
- "weather in Delhi" → {{"function": "get_weather_data", "params": {{"location": "Delhi, India"}}}}
- "weather in Goa" → {{"function": "get_weather_data", "params": {{"location": "Goa, India"}}}}
- "chicken and rice recipes" → {{"function": "search_by_ingredients", "params": {{"ingredients": "chicken,rice"}}}}
- "random recipe" → {{"function": "get_random_recipe", "params": {{}}}}
- "McDonald's info" → {{"function": "search_restaurants", "params": {{"query": "McDonald's"}}}}
- "pizza nutrition" → {{"function": "get_nutrition_info", "params": {{"food_item": "pizza"}}}}
- "weather in Goa and burger recipe" → {{"function": "get_weather_data", "params": {{"location": "Goa, India"}}}}

Respond with only the JSON object:"""

    json_data = {
        "model": "accounts/fireworks/models/llama-v3p1-8b-instruct",
        "prompt": prompt,
        "max_tokens": 100,
        "temperature": 0.1
    }

    try:
        response = requests.post("https://api.fireworks.ai/inference/v1/completions", headers=headers, json=json_data)
        result = response.json().get("choices", [{}])[0].get("text", "").strip()

        # Extract JSON from response
        json_match = re.search(r'\{.*\}', result, re.DOTALL)
        if json_match:
            json_str = json_match.group()
            return json.loads(json_str)
        return None
    except Exception as e:
        print(f"[ERROR] LLM analysis failed: {e}")
        return None

def format_response_with_llm(api_data, user_query, api_function):
    """Use LLM to format response from API data"""
    headers = {
        "Authorization": f"Bearer {FIREWORKS_API_KEY}",
        "Content-Type": "application/json"
    }

    # Truncate API data if too large
    api_data_str = json.dumps(api_data, indent=2)
    if len(api_data_str) > 2500:
        api_data_str = api_data_str[:2500] + "...[truncated]"

    prompt = f"""Format this API response into a friendly, conversational reply with emojis.

CRITICAL: Your response should be ONLY the conversational reply - no notes, no code, no explanations, no extra text.

User's Question: "{user_query}"
API Function Called: {api_function}
API Response: {api_data_str}

Instructions:
- Be conversational and helpful
- Use appropriate emojis
- For recipes: Include title, cooking time, ingredients, and brief instructions
- For weather: Include temperature, conditions, and location
- For restaurants: Include name, cuisine type, and key details
- For nutrition: Include calories and key nutrients
- Use **bold** for headings
- If multiple results, separate with lines (──────)
- If no results found, give a helpful "not found" message
- STOP after the conversational response - no additional notes or code

Your response:"""

    json_data = {
        "model": "accounts/fireworks/models/llama-v3p1-8b-instruct",
        "prompt": prompt,
        "max_tokens": 300,
        "temperature": 0.4
    }

    try:
        response = requests.post("https://api.fireworks.ai/inference/v1/completions", headers=headers, json=json_data)
        if response.status_code == 200:
            result = response.json().get("choices", [{}])[0].get("text", "").strip()

            # Clean up the response - remove any extra content after the main response
            lines = result.split('\n')
            clean_lines = []

            for line in lines:
                # Stop at common patterns that indicate extra content
                if any(pattern in line.lower() for pattern in ['note:', 'here is', 'import ', 'def ', '```', 'api_key']):
                    break
                clean_lines.append(line)

            cleaned_result = '\n'.join(clean_lines).strip()
            return cleaned_result if cleaned_result else "Sorry, I couldn't generate a proper response."
        else:
            return f"Sorry, there was an issue formatting the response."
    except Exception as e:
        return f"Sorry, I couldn't format the response properly. Error: {str(e)}"

# =============================================================================
# API FUNCTIONS (keeping the same)
# =============================================================================

def search_recipes(query, number=2):
    """Search for recipes using Spoonacular API"""
    search_url = "https://api.spoonacular.com/recipes/complexSearch"
    search_params = {
        "apiKey": SPOONACULAR_API_KEY,
        "query": query,
        "number": number,
        "addRecipeInformation": True,
        "fillIngredients": True
    }

    try:
        search_response = requests.get(search_url, params=search_params)
        search_data = search_response.json()

        if "results" not in search_data or not search_data["results"]:
            return {"error": "No recipes found", "results": []}

        # Get detailed information for each recipe
        detailed_recipes = []
        for recipe in search_data["results"]:
            recipe_id = recipe["id"]
            detail_url = f"https://api.spoonacular.com/recipes/{recipe_id}/information"
            detail_params = {"apiKey": SPOONACULAR_API_KEY}

            detail_response = requests.get(detail_url, params=detail_params)
            if detail_response.status_code == 200:
                detailed_recipes.append(detail_response.json())

        return {"results": detailed_recipes}
    except Exception as e:
        return {"error": f"API call failed: {str(e)}"}

def search_by_ingredients(ingredients):
    """Find recipes based on available ingredients"""
    search_url = "https://api.spoonacular.com/recipes/findByIngredients"
    search_params = {
        "apiKey": SPOONACULAR_API_KEY,
        "ingredients": ingredients,
        "number": 3,
        "ranking": 1,
        "ignorePantry": True
    }

    try:
        search_response = requests.get(search_url, params=search_params)
        recipes_data = search_response.json()

        if not recipes_data:
            return {"error": "No recipes found with those ingredients"}

        # Get detailed information for each recipe
        detailed_recipes = []
        for recipe in recipes_data[:2]:  # Limit to 2 recipes
            recipe_id = recipe["id"]
            detail_url = f"https://api.spoonacular.com/recipes/{recipe_id}/information"
            detail_params = {"apiKey": SPOONACULAR_API_KEY}

            detail_response = requests.get(detail_url, params=detail_params)
            if detail_response.status_code == 200:
                detailed_recipe = detail_response.json()
                detailed_recipe['usedIngredients'] = recipe.get('usedIngredients', [])
                detailed_recipe['missedIngredients'] = recipe.get('missedIngredients', [])
                detailed_recipes.append(detailed_recipe)

        return {"results": detailed_recipes}
    except Exception as e:
        return {"error": f"API call failed: {str(e)}"}

def get_random_recipe():
    """Get a random recipe from Spoonacular API"""
    url = "https://api.spoonacular.com/recipes/random"
    params = {"apiKey": SPOONACULAR_API_KEY, "number": 1}

    try:
        response = requests.get(url, params=params)
        return response.json()
    except Exception as e:
        return {"error": f"API call failed: {str(e)}"}

def search_restaurants(query):
    """Search for restaurant information"""
    url = "https://api.spoonacular.com/food/restaurants/search"
    params = {
        "apiKey": SPOONACULAR_API_KEY,
        "query": query,
        "offset": 0,
        "number": 3
    }

    try:
        response = requests.get(url, params=params)
        return response.json()
    except Exception as e:
        return {"error": f"API call failed: {str(e)}"}

def get_nutrition_info(food_item):
    """Get nutritional information for a food item"""
    url = "https://api.spoonacular.com/recipes/guessNutrition"
    params = {"apiKey": SPOONACULAR_API_KEY, "title": food_item}

    try:
        response = requests.get(url, params=params)
        return response.json()
    except Exception as e:
        return {"error": f"API call failed: {str(e)}"}

def get_weather_data(location):
    """Get weather data from Weatherstack API"""
    url = "http://api.weatherstack.com/current"
    params = {
        "access_key": WEATHERSTACK_API_KEY,
        "query": location,
        "units": "m"  # Metric units
    }

    try:
        response = requests.get(url, params=params)
        data = response.json()

        if "error" in data:
            return {"error": f"Weather data not found for {location}"}

        return data
    except Exception as e:
        return {"error": f"API call failed: {str(e)}"}

# =============================================================================
# MAIN EXECUTION ENGINE
# =============================================================================

def execute_function(function_name, params):
    """Execute the appropriate function based on LLM analysis"""
    function_map = {
        "search_recipes": search_recipes,
        "search_by_ingredients": search_by_ingredients,
        "get_random_recipe": get_random_recipe,
        "search_restaurants": search_restaurants,
        "get_nutrition_info": get_nutrition_info,
        "get_weather_data": get_weather_data
    }

    if function_name not in function_map:
        return {"error": f"Unknown function: {function_name}"}

    try:
        function = function_map[function_name]
        if params:
            return function(**params)
        else:
            return function()
    except Exception as e:
        return {"error": f"Function execution failed: {str(e)}"}

def main():
    print("🤖 **Universal Assistant** - LLM-Driven Food & Weather Bot")
    print("=" * 60)
    print("I can help you with:")
    print("🍽️ **Food:** recipes, cooking tips, restaurant info, nutrition")
    print("🌦️ **Weather:** current conditions anywhere in the world")
    print("\nJust ask naturally! Examples:")
    print("• 'Show me some pasta recipes'")
    print("• 'What can I cook with chicken and tomatoes?'")
    print("• 'How's the weather in Tokyo?'")
    print("• 'Tell me about McDonald's'")
    print("• 'Give me a random recipe'")
    print("• 'What's the nutrition info for pizza?'")
    print("\n💡 **Tip:** For multiple requests, ask them separately for best results!")
    print("\nType 'exit' to quit.\n")

    while True:
        user_input = input("You 🧑: ").strip()

        if user_input.lower() in ['exit', 'quit', 'bye', 'stop']:
            print("Bot 🤖: Goodbye! Have a great day! 🌟")
            break

        if not user_input:
            print("Bot 🤖: Please ask me something! 😊")
            continue

        # Check if API keys are configured
        if not FIREWORKS_API_KEY or not SPOONACULAR_API_KEY or not WEATHERSTACK_API_KEY:
            print("Bot 🤖: ⚠️ Please configure your API keys first!")
            continue

        try:
            # Step 1: Let LLM analyze query and determine function to call
            action = analyze_query_and_get_action(user_input)

            if not action or "function" not in action:
                print("Bot 🤖: I'm not sure how to help with that. Try asking about food or weather! 🤔")
                continue

            function_name = action["function"]
            params = action.get("params", {})

            # Optional: Remove debug print or make it conditional
            # print(f"[DEBUG] Calling: {function_name}({params})")

            # Step 2: Execute the function
            api_data = execute_function(function_name, params)

            # Step 3: Check for errors
            if isinstance(api_data, dict) and "error" in api_data:
                print(f"Bot 🤖: {api_data['error']} 😅")
                continue

            # Step 4: Format response using LLM
            response = format_response_with_llm(api_data, user_input, function_name)
            print(f"Bot 🤖: {response}")

        except Exception as e:
            print("Bot 🤖: Oops! Something went wrong while processing your request. 😵")
            print(f"[ERROR] {e}")

if __name__ == "__main__":
    main()

🤖 **Universal Assistant** - LLM-Driven Food & Weather Bot
I can help you with:
🍽️ **Food:** recipes, cooking tips, restaurant info, nutrition
🌦️ **Weather:** current conditions anywhere in the world

Just ask naturally! Examples:
• 'Show me some pasta recipes'
• 'What can I cook with chicken and tomatoes?'
• 'How's the weather in Tokyo?'
• 'Tell me about McDonald's'
• 'Give me a random recipe'
• 'What's the nutrition info for pizza?'

💡 **Tip:** For multiple requests, ask them separately for best results!

Type 'exit' to quit.

You 🧑: What's the weather in Bihar
Bot 🤖: **Weather in Bihar 🌞**
The current temperature in Bihar, India is **29°C** 🌡️. It's a clear night 🌃 with a gentle breeze from the east 🌬️. The humidity is moderate at 68% 🌿. Enjoy the peaceful evening! 😌
You 🧑: how can I make french fries
Bot 🤖: **French Fries Recipe Found! 🍟**

Here's a simple recipe for Sweet Potato Oven Fries:

Sweet Potato Oven Fries 🍟
---------------------------

Cooking Time: 40 minutes
 Servings: 